In [1]:
# !pip install opencv-python-headless==4.5.2.52
# !pip install wrapt
# !pip install mtcnn
# from mtcnn import MTCNN
# import cv2
# import matplotlib.pyplot as plt
import os
import random
import zipfile
import glob
import re
from pathlib import Path
import pandas as pd
from distutils.dir_util import copy_tree
import shutil
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from numpy import load
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import Normalizer
from sklearn.svm import SVC
import subprocess
import sys
import skimage.io as io


In [2]:
!rm -rf /tmp/lfw
!rm -rf /tmp/mlfw
!rm -rf /dataset

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
def extract_lfw_mlfw():
  if(os.path.isdir('/tmp/lfw') == False):
    cmd = "tar -xvf gdrive/MyDrive/datasets/lfw.tar -C /tmp/"
    returned_value = os.system(cmd)
  if(os.path.isdir('/tmp/lfw-deepfunneled') == True):
    os.rename('/tmp/lfw-deepfunneled','/tmp/lfw')

  if(os.path.isdir('/tmp/mlfw/')== False):
    zip_ref = zipfile.ZipFile("/content/gdrive/MyDrive/datasets/MLFW.zip", 'r')
    zip_ref.extractall("/tmp/mlfw/")
    zip_ref.close()

In [5]:
def pick_images_lfw():
  lfw_images = []
  lfw_train = sorted(os.listdir("/tmp/lfw/"))
  for directory in lfw_train:
    dir_path = "/tmp/lfw/" + directory
    no_of_files = len(os.listdir(dir_path)) 
    if(no_of_files >= 3 and no_of_files <= 5):
      lfw_images.append(directory)
  return lfw_images;

In [6]:
def pick_images_mlfw():
  file_list = glob.glob("/tmp/mlfw/origin/*.jpg")
  final_name_list = []
  for file in file_list:
      file_name = Path(file).name
      final_str = ""
      tokens = file_name.split("_")
      for token in tokens:
          if(token.isnumeric()):
              break;
          else:
              final_str += token
              final_str += "_"
      final_name_list.append(final_str)  
  print(len(set(final_name_list)))
  df = pd.DataFrame(final_name_list)
  df.columns = ['name']
  new_df = df.groupby('name').filter(lambda x : (len(x) >= 3 and len(x) <= 5))
  mlfw_images = new_df.values.tolist()
  mlfw_images = [sub[0][ : -1] for sub in mlfw_images]
  return list(set(mlfw_images))


In [7]:
def create_common_dir_list():
  extract_lfw_mlfw();
  lfw_dir = pick_images_lfw();
  mlfw_dir = pick_images_mlfw()
  common_set = set(lfw_dir) & set(mlfw_dir)
  return list(common_set)

In [8]:
def pick_classes(no_of_classes):
  final_list = []
  filtered_list =  create_common_dir_list()
  print("filtered_list_length = " + str(len(filtered_list)))
  for i in range(0, no_of_classes):
      final_list.append(random.choice(filtered_list))
  return final_list

In [9]:
def create_train_test_set_from_lfw_mlfw():
  final_class_list = pick_classes(10)
  print(final_class_list)
  if(os.path.isdir('/tmp/dataset/train') == True):
    shutil.rmtree('/tmp/dataset/train')
  if(os.path.isdir('/tmp/dataset/test') == True):
    shutil.rmtree('/tmp/dataset/test')
  if(os.path.isdir('/tmp/dataset/')  == False):
    os.mkdir('/tmp/dataset/')
  if(os.path.isdir('/tmp/dataset/train')== False):
    os.mkdir('/tmp/dataset/train')
  if(os.path.isdir('/tmp/dataset/test')== False):
    os.mkdir('/tmp/dataset/test')

  for dir in final_class_list:  
    os.system("cp -r " + " /tmp/lfw/"+ dir + " /tmp/dataset/train")
  
  mlfw_jpg_list = []
  for path, subdirs, files in os.walk('/tmp/mlfw/aligned/'):
      for name in files:
          mlfw_jpg_list.append(os.path.join(path, name))

  for dir in final_class_list:
    if(os.path.isdir('/tmp/dataset/test/'+dir)  == False):
      os.mkdir('/tmp/dataset/test/'+dir)
    name_list = [k for k in mlfw_jpg_list if dir in k]
    for nme in name_list:
      shutil.copy(nme, "/tmp/dataset/test/"+dir + "/")

In [10]:
!pip install mtcnn

     |████████████████████████████████| 2.3 MB 5.8 MB/s 


In [11]:
from mtcnn.mtcnn import MTCNN
from numpy import asarray
import cv2
from PIL import Image
 
# extract a single face from a given photograph
def process_face(filename, required_size=(160, 160)):
  # load image from file
  image = Image.open(filename)
  # convert to RGB, if needed
  image = image.convert('RGB')
  # convert to array
  pixels = asarray(image)
  # create the detector, using default weights
  detector = MTCNN()
  # detect faces in the image
  results = detector.detect_faces(pixels)
  if(results):
    # extract the bounding box from the first face
    x1, y1, width, height = results[0]['box']
    # bug fix
    x1, y1 = abs(x1), abs(y1)
    x2, y2 = x1 + width, y1 + height
    # extract the face
    face = pixels[y1:y2, x1:x2]
    # resize pixels to the model size
    image = Image.fromarray(face)
    image = image.resize(required_size)
    face_array = asarray(image)
    cv2.imwrite(filename, face_array)

In [12]:
def get_image_list_from_folder(path):
  final_list = []
  for path, subdirs, files in os.walk(path):
    for name in files:
        final_list.append(os.path.join(path, name))
  return sorted(final_list)

def process_faces(image_list):
  for file in image_list:
    process_face(file)

In [13]:
def plot_images(w, h, fig_w, fig_h, image_list, columns, rows):
  fig = plt.figure(figsize=(fig_w, fig_h))
  for i in range(1, columns*rows +1):
      img = mpimg.imread(image_list[i])
      fig.add_subplot(rows, columns, i)
      plt.title(os.path.basename(image_list[i]))
      plt.imshow(img)
  plt.show()

In [14]:
create_train_test_set_from_lfw_mlfw()

2996
filtered_list_length = 156
['Larry_Thompson', 'Scott_McClellan', 'Bridgette_Wilson-Sampras', 'James_Cunningham', 'Rachel_Griffiths', 'Angelo_Reyes', 'Leslie_Caldwell', 'David_Stern', 'Ethan_Hawke', 'Woody_Allen']


In [15]:
!ls -l /tmp/

total 244
-rw-r--r--    1 root root   1581 May  2 11:46 dap_multiplexer.b1dd50a8fdc0.root.log.INFO.20220502-114640.42
lrwxrwxrwx    1 root root     61 May  2 11:46 dap_multiplexer.INFO -> dap_multiplexer.b1dd50a8fdc0.root.log.INFO.20220502-114640.42
drwxr-xr-x    4 root root   4096 May  2 11:52 dataset
srwxr-xr-x    1 root root      0 May  2 11:46 debugger_qsdga688r
-rw-r--r--    1 root root   1632 May  2 11:52 directoryprefetcher_binary.b1dd50a8fdc0.root.log.INFO.20220502-115209.265
lrwxrwxrwx    1 root root     73 May  2 11:52 directoryprefetcher_binary.INFO -> directoryprefetcher_binary.b1dd50a8fdc0.root.log.INFO.20220502-115209.265
srw-------    1 root root      0 May  2 11:52 drivefs_ipc.0
srw-------    1 root root      0 May  2 11:52 drivefs_ipc.0_shell
drwx------    2 root root   4096 May  2 11:52 initgoogle_syslog_dir.0
drwxr-xr-x 5751 root root 196608 May  2 11:52 lfw
drwxr-xr-x    4 root root   4096 May  2 11:52 mlfw
drwx------    2 root root   4096 May  2 11:52 pyright-293-V

In [16]:
# import math 
train_list = get_image_list_from_folder('/tmp/dataset/train/')
# process_faces(train_list)
# train_len = int(len(train_list))
# per_row = int(math.sqrt(train_len))
plot_images(10, 10, 50, 50, train_list, 5, 5)
print(train_list)

Output hidden; open in https://colab.research.google.com to view.

In [17]:
!pip install dlib

In [18]:
!python /content/gdrive/MyDrive/mfr_test_repo/MaskTheFace/mask_the_face.py --path /tmp/dataset/train/Scott_McClellan/Scott_McClellan_0004.jpg --mask_type all --verbose --write_original_image

Traceback (most recent call last):
  File "/content/gdrive/MyDrive/mfr_test_repo/MaskTheFace/mask_the_face.py", line 7, in <module>
    from utils.aux_functions import *
  File "/content/gdrive/MyDrive/mfr_test_repo/MaskTheFace/utils/aux_functions.py", line 9, in <module>
    from utils.read_cfg import read_cfg
  File "/content/gdrive/MyDrive/mfr_test_repo/MaskTheFace/utils/read_cfg.py", line 6, in <module>
    from dotmap import DotMap
ModuleNotFoundError: No module named 'dotmap'


In [19]:
train_list = get_image_list_from_folder('/tmp/dataset/train/')
process_faces(train_list)
# train_len = int(len(train_list))
# per_row = int(math.sqrt(train_len))
%cd /content/gdrive/MyDrive/mfr_test_repo/MaskTheFace/
!pwd
# !pip install -r requirements.txt
!pip install dotmap
for path in Path('/tmp/dataset/train/').rglob('*.jpg'):
  print ("generating for path" , path)
  result = subprocess.run(["python", "/content/gdrive/MyDrive/mfr_test_repo/MaskTheFace/mask_the_face.py",'--path', path, '--mask_type', 'all', '--verbose', '--write_original_image'],  stdout=subprocess.PIPE, text=True)   
  print(result.stdout)
train_list = get_image_list_from_folder('/tmp/dataset/train/')
plot_images(10, 10, 50, 50, train_list, 5, 5)
print(train_list)

Output hidden; open in https://colab.research.google.com to view.

In [20]:
test_list = get_image_list_from_folder('/tmp/dataset/test/')
process_faces(test_list)
plot_images(10, 10, 50, 50, test_list, 5, 5)

Output hidden; open in https://colab.research.google.com to view.

In [21]:
def get_data_list(path):
  img_list = get_image_list_from_folder(path)
  all_images = []
  for file in img_list:
      img = cv2.imread(file)
      img = cv2.resize(img,(160,160),3)
      all_images.append(img)
  final_name_list = []
  for file in img_list:
      file_name = Path(file).name
      final_str = ""
      tokens = file_name.split("_")
      for token in tokens:
          token = token.split(".",1)[0]
          if(token.isnumeric()):
              break;
          else:
              final_str += token
              final_str += "_"
      final_name_list.append(final_str) 
  final_name_list = [sub[ : -1] for sub in final_name_list]
  return all_images, final_name_list

In [22]:
def create_dataset():
  newTrainX, trainy = get_data_list('/tmp/dataset/train/')
  newTrainX = asarray(newTrainX)
  newTestX, testy = get_data_list('/tmp/dataset/test/')
  newTestX = asarray(newTestX)
  return newTrainX, asarray(trainy), newTestX, asarray(testy)

In [23]:
train_images, train_labels, test_images, test_labels = create_dataset()

import numpy as np
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder

# define example

label_encoder = LabelEncoder()
train_labels = label_encoder.fit_transform(train_labels)
# one hot encode
encoded_train = to_categorical(train_labels)

test_labels = label_encoder.fit_transform(test_labels)
# one hot encode
encoded_test = to_categorical(test_labels)
train_images, test_images = train_images / 255.0, test_images / 255.0
train_images

array([[[[0.50196078, 0.54509804, 0.5372549 ],
         [0.4627451 , 0.50588235, 0.49803922],
         [0.4       , 0.43529412, 0.42352941],
         ...,
         [0.27843137, 0.33333333, 0.31764706],
         [0.26666667, 0.32156863, 0.30588235],
         [0.25882353, 0.31372549, 0.29803922]],

        [[0.48627451, 0.52941176, 0.52156863],
         [0.45098039, 0.49411765, 0.48235294],
         [0.4       , 0.43529412, 0.42352941],
         ...,
         [0.30196078, 0.35686275, 0.34117647],
         [0.28627451, 0.34117647, 0.3254902 ],
         [0.2745098 , 0.32941176, 0.31372549]],

        [[0.4627451 , 0.49803922, 0.48627451],
         [0.43529412, 0.47058824, 0.45882353],
         [0.38823529, 0.42352941, 0.41176471],
         ...,
         [0.3372549 , 0.39215686, 0.37647059],
         [0.31764706, 0.37254902, 0.35686275],
         [0.30196078, 0.35686275, 0.34117647]],

        ...,

        [[0.17647059, 0.25098039, 0.23921569],
         [0.17647059, 0.25098039, 0.23921569]

In [24]:
import tensorflow as tf

from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
def run_ccn_model_1():
  model = models.Sequential()
  # model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(160, 160, 3)))
  model.add(tf.keras.layers.Conv2D(input_shape = (160,160,3), filters = 32, kernel_size = (3,3),activation = "relu", padding = "same" ))
  model.add(layers.MaxPooling2D((2, 2)))
  model.add(layers.Conv2D(64, (3, 3), activation='relu'))
  model.add(layers.MaxPooling2D((2, 2)))
  model.add(layers.Conv2D(64, (3, 3), activation='relu'))
  model.add(layers.Flatten())
  model.add(layers.Dense(64, activation='relu'))
  model.add(layers.Dense(10))
  model.compile(optimizer='adam',
                loss='categorical_crossentropy',
                metrics=['accuracy'],run_eagerly=True)
  model.summary()

  history = model.fit(train_images, encoded_train, epochs=100, 
                      validation_data=(test_images, encoded_test))

  plt.plot(history.history['accuracy'], label='accuracy')
  plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
  plt.xlabel('Epoch')
  plt.ylabel('Accuracy')
  plt.ylim([0.5, 1])
  plt.legend(loc='lower right')

  test_loss, test_acc = model.evaluate(test_images,  encoded_test, verbose=2)
  print(test_acc)

In [25]:
#filters= the depth of output image or kernels
from sklearn.model_selection import train_test_split

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import accuracy_score
from keras.utils import np_utils
import itertools
import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from keras.callbacks import TensorBoard

def run_ccn_model_2(im_shape, x_train, y_train, x_test, y_test):
  # x_train, x_valid, y_train, y_valid=  train_test_split(
  #   x_train, y_train, test_size=.05, random_state=1234,)
  # cnn_model= Sequential([
  #     Conv2D(filters=36, kernel_size=7, activation='relu', input_shape= im_shape),
  #     MaxPooling2D(pool_size=2),
  #     Conv2D(filters=54, kernel_size=5, activation='relu', input_shape= im_shape),
  #     MaxPooling2D(pool_size=2),
  #     Flatten(),
  #     Dense(2024, activation='relu'),
  #     Dropout(0.5),
  #     Dense(1024, activation='relu'),
  #     Dropout(0.5),
  #     Dense(512, activation='relu'),
  #     Dropout(0.5),
  #     #20 is the number of outputs
  #     Dense(9, activation='softmax')  
  # ])
    model = Sequential()
    model.add(Conv2D(64, kernel_size=3, activation='relu', input_shape=im_shape))
    model.add(BatchNormalization()) #----------------
    model.add(Conv2D(64, kernel_size=3, activation='relu'))
    model.add(BatchNormalization()) #----------------
    model.add(Conv2D(64, kernel_size=5, padding='same', activation='relu'))
    model.add(BatchNormalization()) #----------------
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2)) #----------------

    model.add(Conv2D(128, kernel_size=3, activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(128, kernel_size=3, activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(128, kernel_size=5, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))

    model.add(Conv2D(256, kernel_size=3, activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))

    model.add(Flatten())
    model.add(Dense(256))
    model.add(BatchNormalization())
    model.add(Dense(128))
    model.add(BatchNormalization())
    model.add(Dense(9, activation='softmax'))

    model.compile(
        loss='categorical_crossentropy',
        optimizer=Adam(lr=0.0001),
        metrics=['accuracy']
    )

    # cnn_model.compile(optimizer='adam',loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])

    history=model.fit(
      x_train, y_train, batch_size=512,
      epochs=250, verbose=2)
    
    scor = model.evaluate( x_test,  y_test, verbose=0)

    print('test los {:.4f}'.format(scor[0]))
    print('test acc {:.4f}'.format(scor[1]))
    # list all data in history
    print(history.history.keys())
    # summarize history for accuracy
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['loss'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()
    # summarize history for loss
    plt.plot(history.history['loss'])
    plt.plot(history.history['accuracy'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()

In [26]:
run_ccn_model_2( (160,160,3), train_images, encoded_train, test_images, encoded_test)

NameError: ignored

In [40]:
from sklearn.metrics import confusion_matrix
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
import tensorflow as tf

from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import BatchNormalization
from keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import ReLU
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

def run_ccn_model_3(im_shape, X_train, y_train, X_test, y_test):
  train_datagen = ImageDataGenerator(rescale=1./255.,
                                    rotation_range=10,
                                    width_shift_range=0.25,
                                    height_shift_range=0.25,
                                    shear_range=0.1,
                                    zoom_range=0.25,
                                    horizontal_flip=False)

  valid_datagen = ImageDataGenerator(rescale=1./255.)
  from datetime import datetime
  num_classes = 4
  model_name = 'Face_trained_model_'+datetime.now().strftime("%H_%M_%S_")
  model = Sequential(name = model_name)

  model.add(Conv2D(64, kernel_size=3, activation='relu', input_shape=im_shape))
  model.add(BatchNormalization()) #----------------
  model.add(Conv2D(64, kernel_size=3, activation='relu'))
  model.add(BatchNormalization()) #----------------
  model.add(Conv2D(64, kernel_size=5, padding='same', activation='relu'))
  model.add(BatchNormalization()) #----------------
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(0.2)) #----------------

  model.add(Conv2D(128, kernel_size=3, activation='relu'))
  model.add(BatchNormalization())
  model.add(Conv2D(128, kernel_size=3, activation='relu'))
  model.add(BatchNormalization())
  model.add(Conv2D(128, kernel_size=5, padding='same', activation='relu'))
  model.add(BatchNormalization())
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(0.2))

  model.add(Conv2D(256, kernel_size=3, activation='relu'))
  model.add(BatchNormalization())
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(0.2))

  model.add(Flatten())
  model.add(Dense(256))
  model.add(BatchNormalization())
  model.add(Dense(128))
  model.add(BatchNormalization())
  model.add(Dense(9, activation='softmax'))

  model.summary()
  learning_rate = 0.001
  optimizer = RMSprop(lr=learning_rate)

  model.compile(loss='categorical_crossentropy',optimizer=optimizer,metrics=['accuracy'],  run_eagerly=True)
  learning_rate_reduction = ReduceLROnPlateau(monitor='val_loss', 
                                              patience=200,
                                              verbose=1,
                                              factor=0.2)

  ch = ModelCheckpoint('models/'+model_name+'.h5', monitor='val_acc', verbose=0, save_best_only=True, mode='max')

  es = EarlyStopping(monitor='val_loss', mode='min', verbose=0, patience=200)

  tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="logs/"+datetime.now().strftime("%Y%m%d-%H%M%S"))
  epochs = 50
  batch_size = 256
  history = model.fit_generator(train_datagen.flow(X_train, y_train, batch_size=batch_size),
                                steps_per_epoch= X_train.shape[0]//batch_size,
                                epochs=epochs,
                                validation_data=valid_datagen.flow(X_test, y_test),
                                validation_steps=50,
                                verbose = 1,
                                callbacks=[learning_rate_reduction, es, ch, tensorboard_callback])
  loss, acc = model.evaluate(valid_datagen.flow(X_test, y_test))

  print(f'Loss: {loss}\nAccuracy: {acc*100}')

  # Plot training & test accuracy values
  plt.plot(history.history['acc'])
  plt.plot(history.history['val_acc'])
  plt.title('Model accuracy')
  plt.ylabel('Accuracy')
  plt.xlabel('Epoch')
  plt.legend(['Train', 'Test'], loc='upper left')
  plt.show()

  # Plot training & test loss values
  plt.plot(history.history['loss'])
  plt.plot(history.history['val_loss'])
  plt.title('Model loss')
  plt.ylabel('Loss')
  plt.xlabel('Epoch')
  plt.legend(['Train', 'Test'], loc='upper left')
  plt.show()
  # Load the TensorBoard notebook extension
  %load_ext tensorboard
  %tensorboard --logdir logs/
  from sklearn.metrics import classification_report

  print(classification_report(np.argmax(y_test, axis=1), np.argmax(model.predict(X_test),axis=1)))
  from sklearn.metrics import confusion_matrix,f1_score, precision_score, recall_score
  import seaborn as sn
  y_true = np.argmax(y_test, axis=1)
  y_pred = np.argmax(model.predict(X_test), axis=1)
  data = confusion_matrix(y_true, y_pred)
  df_cm = pd.DataFrame(data, columns=np.unique(y_true), index = np.unique(y_true))
  df_cm.index.name = 'Actual'
  df_cm.columns.name = 'Predicted'
  plt.figure(figsize = (10,7))
  sn.set(font_scale=1.4)#for label size
  sn.heatmap(df_cm, cmap="Blues", annot=True,annot_kws={"size": 16})# font size
  print(f1_score(y_true, y_pred, average="macro")*100)
  print(precision_score(y_true, y_pred, average="macro")*100)
  print(recall_score(y_true, y_pred, average="macro")*100) 
  sn.heatmap(df_cm, annot=True, fmt="d", cmap="plasma");

In [35]:
print(encoded_train)

[[1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]]


In [41]:
run_ccn_model_3( (160,160,3), train_images, encoded_train, test_images, encoded_test)

Model: "Face_trained_model_11_47_37_"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_945 (Conv2D)         (None, 158, 158, 64)      1792      
                                                                 
 batch_normalization_36 (Bat  (None, 158, 158, 64)     256       
 chNormalization)                                                
                                                                 
 conv2d_946 (Conv2D)         (None, 156, 156, 64)      36928     
                                                                 
 batch_normalization_37 (Bat  (None, 156, 156, 64)     256       
 chNormalization)                                                
                                                                 
 conv2d_947 (Conv2D)         (None, 156, 156, 64)      102464    
                                                                 
 batch_normalization_38 (Bat  (None, 1

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:87: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.


Epoch 1/50


ValueError: ignored